In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Setting

### 1-1 Data set

We use the Wine Quality dataset, which is available in the TensorFlow Datasets. We use the red wine subset, which contains 4,898 examples. The dataset has 11numerical physicochemical features of the wine, and the task is to predict the wine quality, which is a score between 0 and 10. In this example, we treat this as a regression task.

You can install TensorFlow Datasets using the following command:

In [2]:
pip install tensorflow-probability


Note: you may need to restart the kernel to use updated packages.


### 1-2 Setup

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

### 1-3 Create training and evaluation datasets


In [4]:
def get_train_and_test_splits(train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because th dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset

### 1-4 Compile, train, and evaluate the model


In [5]:
hidden_units = [8, 8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

### 1-5 Create model inputs


In [6]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs


# 2. Experiment

### 2-1 Experiment 1: standard neural network


We create a standard deterministic neural network model as a baseline.



In [7]:
def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

Let's split the wine dataset into training and test sets, with 85% and 15% of the examples, respectively.

In [8]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)

2022-03-30 07:52:59.151170: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "Not found: Could not locate the credentials file.". Retrieving token from GCE failed with "Failed precondition: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/4898 [00:00<?, ? examples/s]

Shuffling wine_quality-train.tfrecord...:   0%|          | 0/4898 [00:00<?, ? examples/s]

Dataset wine_quality downloaded and prepared to /root/tensorflow_datasets/wine_quality/white/1.0.0. Subsequent calls will reuse this data.


2022-03-30 07:53:05.875771: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Now let's train the baseline model. We use the MeanSquaredError as the loss function.



In [9]:
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

Start training the model...
Epoch 1/100


2022-03-30 07:53:06.979126: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


16/17 [===========================>..] - ETA: 0s - loss: 39.4661 - root_mean_squared_error: 6.2822 

2022-03-30 07:53:07.698753: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2022-03-30 07:53:07.701968: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


17/17 [==============================] - 2s 55ms/step - loss: 39.4767 - root_mean_squared_error: 6.2831 - val_loss: 37.6272 - val_root_mean_squared_error: 6.1341
Epoch 2/100
17/17 [==============================] - 0s 7ms/step - loss: 37.4775 - root_mean_squared_error: 6.1219 - val_loss: 35.7860 - val_root_mean_squared_error: 5.9821
Epoch 3/100
17/17 [==============================] - 0s 7ms/step - loss: 35.7788 - root_mean_squared_error: 5.9815 - val_loss: 34.0066 - val_root_mean_squared_error: 5.8315
Epoch 4/100
17/17 [==============================] - 0s 7ms/step - loss: 34.1453 - root_mean_squared_error: 5.8434 - val_loss: 32.2837 - val_root_mean_squared_error: 5.6819
Epoch 5/100
17/17 [==============================] - 0s 7ms/step - loss: 32.5508 - root_mean_squared_error: 5.7053 - val_loss: 30.6443 - val_root_mean_squared_error: 5.5357
Epoch 6/100
17/17 [==============================] - 0s 7ms/step - loss: 30.9965 - root_mean_squared_error: 5.5675 - val_loss: 29.0791 - val_root_

We take a sample from the test set use the model to obtain predictions for them. Note that since the baseline model is deterministic, we get a single a point estimate prediction for each test example, with no information about the uncertainty of the model nor the prediction.

In [10]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")

Predicted: 5.4 - Actual: 6.0
Predicted: 6.2 - Actual: 5.0
Predicted: 5.5 - Actual: 6.0
Predicted: 5.4 - Actual: 5.0
Predicted: 6.1 - Actual: 5.0
Predicted: 5.8 - Actual: 6.0
Predicted: 5.6 - Actual: 6.0
Predicted: 6.5 - Actual: 7.0
Predicted: 6.4 - Actual: 6.0
Predicted: 6.0 - Actual: 5.0


### 2-2 Experiment 2: Bayesian neural network (BNN)


The object of the Bayesian approach for modeling neural networks is to capture the epistemic uncertainty, which is uncertainty about the model fitness, due to limited training data.

The idea is that, instead of learning specific weight (and bias) values in the neural network, the Bayesian approach learns weight distributions - from which we can sample to produce an output for a given input - to encode weight uncertainty.

Thus, we need to define prior and the posterior distributions of these weights, and the training process is to learn the parameters of these distributions.

In [11]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model


In [12]:
#We use the tfp.layers.DenseVariational layer 
# instead of the standard keras.layers.Dense layer in the neural network model.

def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

The epistemic uncertainty can be reduced as we increase the size of the training data. That is, the more data the BNN model sees, the more it is certain about its estimates for the weights (distribution parameters). Let's test this behaviour by training the BNN model on a small subset of the training set, and then on the full training set, to compare the output variances.

In [13]:
#Train BNN with a small training subset.

num_epochs = 500
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)

2022-03-30 07:53:30.211301: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Start training the model...
Epoch 1/500
5/5 [==============================] - 3s 223ms/step - loss: 31.5808 - root_mean_squared_error: 5.6185 - val_loss: 28.3234 - val_root_mean_squared_error: 5.3206
Epoch 2/500
5/5 [==============================] - 0s 36ms/step - loss: 37.9602 - root_mean_squared_error: 6.1601 - val_loss: 39.4523 - val_root_mean_squared_error: 6.2799
Epoch 3/500
5/5 [==============================] - 0s 35ms/step - loss: 34.1592 - root_mean_squared_error: 5.8434 - val_loss: 33.9426 - val_root_mean_squared_error: 5.8247
Epoch 4/500
5/5 [==============================] - 0s 27ms/step - loss: 34.6822 - root_mean_squared_error: 5.8881 - val_loss: 33.7620 - val_root_mean_squared_error: 5.8088
Epoch 5/500
5/5 [==============================] - 0s 24ms/step - loss: 34.8965 - root_mean_squared_error: 5.9061 - val_loss: 28.9582 - val_root_mean_squared_error: 5.3796
Epoch 6/500
5/5 [==============================] - 0s 24ms/step - loss: 31.7651 - root_mean_squared_error: 5.63

Since we have trained a BNN model, the model produces a different output each time we call it with the same input, since each time a new set of weights are sampled from the distributions to construct the network and produce an output. The less certain the mode weights are, the more variability (wider range) we will see in the outputs of the same inputs.

In [14]:

def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {targets[idx]}"
        )


compute_predictions(bnn_model_small)

Predictions mean: 5.42, min: 4.66, max: 5.97, range: 1.31 - Actual: 6.0
Predictions mean: 6.2, min: 5.94, max: 6.43, range: 0.5 - Actual: 5.0
Predictions mean: 5.57, min: 4.75, max: 6.12, range: 1.37 - Actual: 6.0
Predictions mean: 5.6, min: 4.74, max: 6.19, range: 1.45 - Actual: 5.0
Predictions mean: 6.1, min: 5.6, max: 6.36, range: 0.76 - Actual: 5.0
Predictions mean: 5.77, min: 4.72, max: 6.15, range: 1.43 - Actual: 6.0
Predictions mean: 5.66, min: 4.78, max: 6.14, range: 1.36 - Actual: 6.0
Predictions mean: 6.3, min: 6.02, max: 6.49, range: 0.47 - Actual: 7.0
Predictions mean: 6.28, min: 5.98, max: 6.47, range: 0.49 - Actual: 6.0
Predictions mean: 5.98, min: 5.13, max: 6.31, range: 1.18 - Actual: 5.0


Train BNN with the whole training set.


In [15]:
num_epochs = 500
bnn_model_full = create_bnn_model(train_size)
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)

compute_predictions(bnn_model_full)

Start training the model...
Epoch 1/500
17/17 [==============================] - 3s 53ms/step - loss: 50.3613 - root_mean_squared_error: 7.0963 - val_loss: 48.0657 - val_root_mean_squared_error: 6.9326
Epoch 2/500
17/17 [==============================] - 0s 8ms/step - loss: 50.5115 - root_mean_squared_error: 7.1068 - val_loss: 45.0161 - val_root_mean_squared_error: 6.7091
Epoch 3/500
17/17 [==============================] - 0s 8ms/step - loss: 47.4187 - root_mean_squared_error: 6.8858 - val_loss: 47.2975 - val_root_mean_squared_error: 6.8770
Epoch 4/500
17/17 [==============================] - 0s 8ms/step - loss: 44.1294 - root_mean_squared_error: 6.6426 - val_loss: 45.3142 - val_root_mean_squared_error: 6.7312
Epoch 5/500
17/17 [==============================] - 0s 8ms/step - loss: 42.7388 - root_mean_squared_error: 6.5372 - val_loss: 44.6223 - val_root_mean_squared_error: 6.6797
Epoch 6/500
17/17 [==============================] - 0s 8ms/step - loss: 42.2680 - root_mean_squared_error

Notice that the model trained with the full training dataset shows smaller range (uncertainty) in the prediction values for the same inputs, compared to the model trained with a subset of the training dataset.

### 2-3 Experiment 3: probabilistic Bayesian neural network


So far, the output of the standard and the Bayesian NN models that we built is deterministic, that is, produces a point estimate as a prediction for a given example. We can create a probabilistic NN by letting the model output a distribution. In this case, the model captures the aleatoric uncertainty as well, which is due to irreducible noise in the data, or to the stochastic nature of the process generating the data.

In this example, we model the output as a IndependentNormal distribution, with learnable mean and variance parameters. If the task was classification, we would have used IndependentBernoulli with binary classes, and OneHotCategorical with multiple classes, to model distribution of the model output

In [16]:
def create_probablistic_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # Create a probabilisticå output (Normal distribution), and use the `Dense` layer
    # to produce the parameters of the distribution.
    # We set units=2 to learn both the mean and the variance of the Normal distribution.
    distribution_params = layers.Dense(units=2)(features)
    outputs = tfp.layers.IndependentNormal(1)(distribution_params)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

Since the output of the model is a distribution, rather than a point estimate, we use the negative loglikelihood as our loss function to compute how likely to see the true data (targets) from the estimated distribution produced by the model.

In [17]:
def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)


num_epochs = 1000
prob_bnn_model = create_probablistic_bnn_model(train_size)
run_experiment(prob_bnn_model, negative_loglikelihood, train_dataset, test_dataset)

Start training the model...
Epoch 1/1000
17/17 [==============================] - 3s 59ms/step - loss: 35.8911 - root_mean_squared_error: 6.3669 - val_loss: 33.1396 - val_root_mean_squared_error: 5.9624
Epoch 2/1000
17/17 [==============================] - 0s 9ms/step - loss: 27.5632 - root_mean_squared_error: 6.2279 - val_loss: 29.1586 - val_root_mean_squared_error: 6.5523
Epoch 3/1000
17/17 [==============================] - 0s 9ms/step - loss: 26.5869 - root_mean_squared_error: 6.1608 - val_loss: 35.2827 - val_root_mean_squared_error: 5.8919
Epoch 4/1000
17/17 [==============================] - 0s 9ms/step - loss: 24.8013 - root_mean_squared_error: 6.2423 - val_loss: 35.3289 - val_root_mean_squared_error: 5.9146
Epoch 5/1000
17/17 [==============================] - 0s 9ms/step - loss: 18.1666 - root_mean_squared_error: 6.0175 - val_loss: 13.3074 - val_root_mean_squared_error: 5.6927
Epoch 6/1000
17/17 [==============================] - 0s 9ms/step - loss: 15.8049 - root_mean_squared

Now let's produce an output from the model given the test examples. The output is now a distribution, and we can use its mean and variance to compute the confidence intervals (CI) of the prediction.

In [18]:
prediction_distribution = prob_bnn_model(examples)
prediction_mean = prediction_distribution.mean().numpy().tolist()
prediction_stdv = prediction_distribution.stddev().numpy()

# The 95% CI is computed as mean ± (1.96 * stdv)
upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()

for idx in range(sample):
    print(
        f"Prediction mean: {round(prediction_mean[idx][0], 2)}, "
        f"stddev: {round(prediction_stdv[idx][0], 2)}, "
        f"95% CI: [{round(upper[idx][0], 2)} - {round(lower[idx][0], 2)}]"
        f" - Actual: {targets[idx]}"
    )

Prediction mean: 5.08, stddev: 0.66, 95% CI: [6.37 - 3.79] - Actual: 6.0
Prediction mean: 6.19, stddev: 0.78, 95% CI: [7.72 - 4.67] - Actual: 5.0
Prediction mean: 5.65, stddev: 0.71, 95% CI: [7.05 - 4.25] - Actual: 6.0
Prediction mean: 5.63, stddev: 0.7, 95% CI: [7.0 - 4.26] - Actual: 5.0
Prediction mean: 5.88, stddev: 0.75, 95% CI: [7.34 - 4.41] - Actual: 5.0
Prediction mean: 5.62, stddev: 0.7, 95% CI: [7.0 - 4.24] - Actual: 6.0
Prediction mean: 5.71, stddev: 0.75, 95% CI: [7.18 - 4.24] - Actual: 6.0
Prediction mean: 6.62, stddev: 0.85, 95% CI: [8.28 - 4.96] - Actual: 7.0
Prediction mean: 6.47, stddev: 0.83, 95% CI: [8.09 - 4.84] - Actual: 6.0
Prediction mean: 6.17, stddev: 0.77, 95% CI: [7.67 - 4.66] - Actual: 5.0
